<a href="https://colab.research.google.com/github/riccardo247/star-GPT/blob/main/offchart_star_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction:**
This a notebook showing how to generate sentiment with an all in text approach.
The model used is GPT-neo. This specific version was pretrained on the pile and on c4 (colossal dataset) for 700k steps. Further pretrained on yelp reviews and imdb dataset.
For this specific task the model was fine tuned as well on reviews dataset.

**Example:** Given a reviews or text in input the model wil complete with sentiment negative/positive/neutral.

**Review:**
Succeeds in providing a disquiet world the long-dreaded completion of the Police Academy series . sentiment is:...


**Download all needd packages**


> package GPTNeo is needed with all the requirements



In [ ]:
#@title Setup
%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install -q -r requirements.txt

Cloning into 'GPTNeo'...
remote: Enumerating objects: 3747, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 3747 (delta 167), reused 255 (delta 147), pack-reused 3468
Receiving objects: 100% (3747/3747), 1.43 MiB | 11.28 MiB/s, done.
Resolving deltas: 100% (2167/2167), done.
/content/GPTNeo
     |████████████████████████████████| 368kB 6.7MB/s 
     |████████████████████████████████| 14.2MB 297kB/s 
     |████████████████████████████████| 112kB 43.9MB/s 
     |████████████████████████████████| 394.7MB 39kB/s 
     |████████████████████████████████| 3.4MB 45.3MB/s 
     |████████████████████████████████| 2.9MB 50.0MB/s 
     |████████████████████████████████| 1.5MB 45.6MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 2.2MB 47.8MB/s 
     |████████████████████████████████| 184kB 58.6MB/s 
     |████████████████████████████████| 1.0MB 47.1MB/s 
     |██████████████████

**Gopogle authentication** could be needed to access the Google bucket storing weights

In [ ]:
from google.colab import auth
auth.authenticate_user()
!gcloud init

**Configuration**


> Remember to change running time to TPU



> The following are configurations files for the model and data. They do not need to be modified


> It is using GPT2 tokenizer





In [ ]:
%%writefile configs/GPT3_XL_bakeoff.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 1,
    "attn_dropout": 0,
    "train_steps": 744136,
    "lr_decay_end" : 300000,
    "eval_steps": 20,
    "predict_steps":1,
    "res_dropout": 0,
    "eval_batch_size": 1,
    "predict_batch_size": 1,
    "iterations": 100,
    "n_embd": 2048,
    "datasets": [["sst3_shortbake_label_pred"]], 
    "model_path": "gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/all_balanced_train_b",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:1,y:8",
    "layout" : "batch:x,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16",
    "padding_id" : 50257,
    "eos_id" : 50256
}

Overwriting configs/GPT3_XL_bakeoff.json


In [ ]:
%%writefile configs/dataset_configs/sst3_shortbake_label_pred.json
{
    "n_vocab": 50257,
    "path": "gs://bigbird-freefly/sst3/bakeoff_entry//sst*.tfrecords", 
    "eval_path": "gs://bigbird-freefly/sst3/dev//imbd*.tfrecords",
    "tokenizer_is_pretrained": true,
    "tokenizer_path": "gpt2",
    "eos_id" : 50256,
    "padding_id": 50257
}

Overwriting configs/dataset_configs/sst3_shortbake_label_pred.json


In [ ]:
!gsutil cp gs://bigbird-freefly/sst3/files/sample.py ./

Copying gs://bigbird-freefly/sst3/files/sample.py...
- [1 files][  9.1 KiB/  9.1 KiB]                                                
Operation completed over 1 objects/9.1 KiB.                                      


**Review examples**


> The following code cell writes to a txt a review example.  Iprovide 3 examples for positive, neutral, negative sentiment. The text should be formatted in 3 lines as:


> This is the review:
...put the text of the review in this line...
sentiment is:





In [ ]:
%%writefile review_11820-00001.txt
This is a review:
Enough similarities to Gymkata and Howie Long 's Firestorm that my fingernails instinctively crawled towards my long-suffering eyeballs .
sentiment is:

Overwriting review_11820-00001.txt


In [ ]:
%%writefile review_11830-00001.txt
This is a review:
While the mystery surrounding the nature of the boat 's malediction remains intriguing enough to sustain mild interest , the picture refuses to offer much accompanying sustenance in the way of characterization , humor or plain old popcorn fun .
sentiment is:

Overwriting review_11830-00001.txt


In [ ]:
%%writefile review_11821-00001.txt
This is a review:
Succeeds in providing a disquiet world the long-dreaded completion of the Police Academy series .
sentiment is:

**Sentimen prediction:**
The following code is running prediction for the txt file and output the result to the cell. Change the filename or create a new one

>The first time it will take 10 minutes because it has to download the model weights. Second time will be 1-2 minutes. This notebook is just to show how it works but it is very slow!
Output will be the input ending in sentiment like this:
This is a review:
While the mystery surrounding the nature of the boat 's malediction remains intriguing enough to sustain mild interest , the picture refuses to offer much accompanying sustenance in the way of characterization , humor or plain old popcorn fun .
sentiment is: **negative**




In [ ]:
review_file ='review_11820-00001.txt'
!python3 main.py --model GPT3_XL_bakeoff --steps_per_checkpoint 100 --tpu colab --predict --prompt $review_file

2021-04-21 20:45:26.644600: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 736911
Saving config to gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/all_balanced_train_b
2021-04-21 20:45:35.333094: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-21 20:45:35.334145: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-21 20:45:35.346532: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-21 20:45:35.346619: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (945b4dd6c79b): /proc/driver/nvidia/version does not e

**Run full test data prediction**

> All the test set saved in a tfrecord file gs://bigbird-freefly/sst3/bakeoff_entry//sst*.tfrecords


>Prediction result will be saved in /test_pred/test_pred_full.jsonl. Running time is about 40 minutes


> TODO: add here match.py to match results with original ids and compute f1



> Som parameters are still hard-coded in the code. For example sampling temperature is set to 0. I overwrite few files here before execution













In [ ]:
!mkdir test_pred
!gsutil cp gs://bigbird-freefly/sst3/files/main_predict.py ./
!gsutil cp gs://bigbird-freefly/sst3/files/inputs.py ./

Copying gs://bigbird-freefly/sst3/files/main_predict.py...
- [1 files][ 11.3 KiB/ 11.3 KiB]                                                
Operation completed over 1 objects/11.3 KiB.                                     
Copying gs://bigbird-freefly/sst3/files/inputs.py...
/ [1 files][ 21.7 KiB/ 21.7 KiB]                                                
Operation completed over 1 objects/21.7 KiB.                                     


In [ ]:
%%writefile configs/GPT3_XL_bakeoff.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 8,
    "attn_dropout": 0,
    "train_steps": 744136,
    "lr_decay_end" : 300000,
    "eval_steps": 20,
    "predict_steps":1,
    "res_dropout": 0,
    "eval_batch_size": 8,
    "predict_batch_size": 8,
    "iterations": 100,
    "n_embd": 2048,
    "datasets": [["sst3_shortbake_label_pred"]], 
    "model_path": "gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/all_balanced_train_b",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:2,y:4",
    "layout" : "batch:x,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16",
    "padding_id" : 50257,
    "eos_id" : 50256
}

Overwriting configs/GPT3_XL_bakeoff.json


In [ ]:
!python3 main_predict.py --predict --steps_per_checkpoint 200 --tpu colab --model GPT3_XL_bakeoff

In [ ]:
!head ./test_pred/test_pred.jsonl

{"text": "The host would not seat them, said they were pretty booked, but offered them a table in the bar.", "stars": "neutral"}
{"text": "The film is just a big, gorgeous, mind-blowing, breath-taking mess.", "stars": "neutral"}
{"text": "With a story inspired by the tumultuous surroundings of Los Angeles, where feelings of marginalization loom for every dreamer with a burst bubble, The Dogwalker has a few characters and ideas, but it never manages to put them on the same path.", "stars": "neutral"}
{"text": "This was our fourth visit to Blue Bird.", "stars": "neutral"}
{"text": "Apparently there are more rooms (downstairs bar with sports on tv) then we saw.", "stars": "neutral"}
{"text": "A grim, flat and boring werewolf movie that refuses to develop an energy level.", "stars": "negative"}
{"text": "One of those movies where you walk out of the theater not feeling cheated exactly, but feeling pandered to, which, in the end, might be all the more infuriating.", "stars": "negative"}
{"t